<a href="https://colab.research.google.com/github/ktakano/takanolab/blob/master/week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI演習 第6回
### ディープラーニングによる自然言語処理 (1)



### 自然言語処理とは
* 日本語、英語、タイ語など、日常の会話などで使用する言語を自然言語と言います。
自然言語処理とは、自然言語を対象として単語や句の抽出や、係り受けの解析といった処理を経て、コンピュータが機械翻訳、自動要約、文章分類、文脈理解、会話生成などを行えるようにするための処理です。

* 自然言語処理の応用例
 * 文書分類
 * 検索
 * 機械翻訳
 * 文書要約
 * 質問応答
 * 対話
 * 品詞タグ付け
 * 単語分割
 * 語義曖昧性解消
 * 固有表現抽出
 * 構文解析
 * 述語項構造認識



### 自然言語処理とディープラーニング
* トピックモデルによる文書中の単語の出現確率の推定や隠れマルコフモデルによる品詞推定などが行われてきました。

* 2013年にword2vecのような単語の分散表現をニューラルネットワークで学習する方式が考案され、さらにRNNやLSTMが自然言語処理に適用されるとともに、機械翻訳、対話文生成、自動要約、画像説明文の生成などの応用が広がっていきました。

### 言語モデルと文脈
* 単語が文書中に出現する過程を確率過程と見なし、単語がある位置に出現する確率を計算するモデルを言語モデルといいます。また言語モデルにおいて，ある単語の出現確率を計算する際に用いる周囲の単語を文脈といいます。

### 自然言語処理の流れ

1.   データセットの準備
2.   前処理
3.   単語の数値化
4.   データセットの学習 = 応用モデルの構築
5.   応用モデルによる分類や回帰


### 前処理
* 形態素解析、n-gram分割、ストップワードなどの処理をして、文章を解析プログラムが処理しやすい形式に加工することです。

 * 表記の統一
 * 小文字化と大文字化
 * 単語置換



### 形態素解析の実行例

形態素解析ライブラリjanomeを利用した形態素解析の実行例です。最初にjanomeをインストールしています。

In [ ]:
!pip install janome

     |████████████████████████████████| 19.7MB 69.5MB/s 


In [ ]:
from janome.tokenizer import Tokenizer

text = '私は空を見て、あなたは鏡をのぞき込む。'

jt = Tokenizer()
for token in jt.tokenize(text):
  print(token)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
空	名詞,一般,*,*,*,*,空,ソラ,ソラ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
見	動詞,自立,*,*,一段,連用形,見る,ミ,ミ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
、	記号,読点,*,*,*,*,、,、,、
あなた	名詞,代名詞,一般,*,*,*,あなた,アナタ,アナタ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
鏡	名詞,一般,*,*,*,*,鏡,カガミ,カガミ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
のぞき込む	動詞,自立,*,*,五段・マ行,基本形,のぞき込む,ノゾキコム,ノゾキコム
。	記号,句点,*,*,*,*,。,。,。


In [ ]:
for token in jt.tokenize(text, wakati=True):
  print(token)

私
は
空
を
見
て
、
あなた
は
鏡
を
のぞき込む
。


### 単語の数値化
* 単語間の関連性などを定量的に計算できるように、単語を数値化します。


In [ ]:
text = 'I look at sky and you look in mirror.'

In [ ]:
text = text.lower()
text = text.replace('.', ' .')
words = text.split(' ')

In [ ]:
def word2id(words):

  word_to_id = {}
  
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
  
  return word_to_id
  
def id2word(word_to_id):
  id_to_word = {}
  for word, id in word_to_id.items():
    id_to_word[id] = word

  return id_to_word

In [ ]:
word_to_id = word2id(words)
print(word_to_id)

id_to_word = id2word(word_to_id)
print(id_to_word)

{'i': 0, 'look': 1, 'at': 2, 'sky': 3, 'and': 4, 'you': 5, 'in': 6, 'mirror': 7, '.': 8}
{0: 'i', 1: 'look', 2: 'at', 3: 'sky', 4: 'and', 5: 'you', 6: 'in', 7: 'mirror', 8: '.'}


### One-hotベクトル表現
要素の値が0と1からなり、かつ1か所だけが1のベクトルです。自然言語を処理するニューラルネットワークでは、one-hotベクトルで表現されたものを入力とすることが多いです。

'I look at sky and you look in mirror.'<br>
を上の例のように分解した場合、数値を要素番号とすると各単語のone-hotベクトルを下記のように生成することができます。

i: [1, 0, 0, 0, 0, 0, 0, 0, 0] <br>
look: [0, 1, 0, 0, 0, 0, 0, 0, 0] <br>
at: [0, 0, 1, 0, 0, 0, 0, 0, 0] <br>

In [ ]:
def make_one_hot(corpus):
    N = corpus.shape[0]
    dim =len(word_to_id)

    one_hot = np.zeros((N, dim), dtype=np.int32)
    for idx, word_id in enumerate(corpus):
        one_hot[idx, word_id] = 1

    return one_hot


In [ ]:
import numpy as np
corpus = [word_to_id[word] for word in words]
corpus = np.array(corpus)

print(corpus)

make_one_hot(corpus)

[0 1 2 3 4 5 1 6 7 8]


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

### 単語の分散表現（埋め込み表現）
* 単語をベクトルで表現したものです。
* 単語分散表現を得るためのニューラルネットワークを用いた手法として，下記のものがあります。

 * Word2Vec
 * GloVe
 * fastText

* 通常の深層学習を使用して学習させ、単語分散表現を得ることもできます。


### Word2Vec
* 単語の分散表現(埋め込み)を生成する手法

* CBOWモデルとskip gramモデルを使用
* 2013年にTomas Mikolovらが考案
* 分布仮説（単語の意味は周囲の単語によって形成されるという仮説）に基づいて、単語の意味表現を学習する。
* 2層のニューラルネットワークで学習する。隠れ層の重みが単語の分散表現となる。

* 生成された単語の分散表現は、各単語の意味を表したベクトル行列となっており，単語間の距離を計算できる。

 * vector('Paris') - vector('France') + vector('Italy’) = vector(‘Rome’) <br>
 * vector('king') - vector('man') + vector('woman’) = vector('queen')

<br>

#### CBOWモデル

* 複数の単語を文脈として、一つの単語を予測する。文脈単語の順序は問わない。
<br>

<img src='https://drive.google.com/uc?export=view&id=1zGpPVAQ7hBmkjEkeFMNa0p5vHXv192hq' width='60%'>

#### Skip-gramモデル

* 一つの単語を使用してして、複数の単語を文脈として予測する。文脈単語は、入力単語の近さに応じて重みを決める。

<img src='https://drive.google.com/uc?export=view&id=18ipD4BXzaNA9tnggGE8yZTGA0mD70rSB' width='60%'>


### 単語の分散表現行列の作成

下記のサイトで提供されるコーパスを用いて、単語の分散表現行列を作成します。

(英語) http://mattmahoney.net/dc/textdata.html <br>
(日本語) https://github.com/Hironsan/ja.text8

In [ ]:
!wget http://mattmahoney.net/dc/text8.zip

--2020-10-29 12:49:28--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.24
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   337KB/s    in 94s     

2020-10-29 12:51:02 (327 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [ ]:
!unzip text8.zip

Archive:  text8.zip
  inflating: text8                   


In [ ]:
import logging
from gensim.models.word2vec import Word2Vec, Text8Corpus

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = Text8Corpus('text8')
model = Word2Vec(sentences, size=100)

model.save('model.bin')

2020-10-29 12:56:32,385 : INFO : collecting all words and their counts
2020-10-29 12:56:32,402 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-29 12:56:37,377 : INFO : collected 253854 word types from a corpus of 17005207 raw words and 1701 sentences
2020-10-29 12:56:37,381 : INFO : Loading a fresh vocabulary
2020-10-29 12:56:37,604 : INFO : effective_min_count=5 retains 71290 unique words (28% of original 253854, drops 182564)
2020-10-29 12:56:37,605 : INFO : effective_min_count=5 leaves 16718844 word corpus (98% of original 17005207, drops 286363)
2020-10-29 12:56:37,849 : INFO : deleting the raw counts dictionary of 253854 items
2020-10-29 12:56:37,860 : INFO : sample=0.001 downsamples 38 most-common words
2020-10-29 12:56:37,861 : INFO : downsampling leaves estimated 12506280 word corpus (74.8% of prior 16718844)
2020-10-29 12:56:38,169 : INFO : estimated required memory for 71290 words and 100 dimensions: 92677000 bytes
2020-10-29 12:56:38,170 : 

In [ ]:
model = Word2Vec.load('model.bin')

print(model.wv['dog'])
print(model.wv['dog'].shape)
model.wv.most_similar(['car'])

2020-10-29 13:44:44,240 : INFO : loading Word2Vec object from model.bin
2020-10-29 13:44:44,837 : INFO : loading wv recursively from model.bin.wv.* with mmap=None
2020-10-29 13:44:44,838 : INFO : setting ignored attribute vectors_norm to None
2020-10-29 13:44:44,838 : INFO : loading vocabulary recursively from model.bin.vocabulary.* with mmap=None
2020-10-29 13:44:44,839 : INFO : loading trainables recursively from model.bin.trainables.* with mmap=None
2020-10-29 13:44:44,840 : INFO : setting ignored attribute cum_table to None
2020-10-29 13:44:44,841 : INFO : loaded model.bin
2020-10-29 13:44:45,040 : INFO : precomputing L2-norms of word weight vectors


[-3.29781681e-01 -2.19763255e+00  7.30305552e-01  5.65891601e-02
 -8.17861676e-01 -1.90371943e+00  1.26455522e+00 -2.37285331e-01
 -1.24525465e-01  1.10376644e+00 -1.44059277e+00 -1.75673175e+00
 -1.22593246e-01  3.43172818e-01 -1.50801754e+00  9.88996148e-01
  1.49990714e+00  1.00307012e+00  1.17061615e+00  5.34562349e-01
 -1.12042117e+00 -1.35945046e+00 -1.00587153e+00 -4.48431671e-01
 -6.88128352e-01  7.67244220e-01 -1.10500062e+00 -3.77828240e-01
  9.81238961e-01  1.10676743e-01  1.66719943e-01  5.59021682e-02
 -1.36575317e+00  8.31565201e-01  7.43897408e-02 -2.23381925e+00
 -6.58857524e-02  1.53459504e-01  1.00218856e+00  7.57093072e-01
  4.14303355e-02  1.65139842e+00  1.51228309e+00  1.84139416e-01
  1.39303148e+00  6.84117138e-01 -3.13499302e-01  1.05936170e+00
 -1.56172132e+00  3.43215376e-01 -2.32463169e+00  2.44814372e+00
  6.07408822e-01 -1.92601180e+00 -7.36339390e-01 -1.68348920e+00
  2.99419574e-02  4.79800105e-01  1.56553817e+00 -7.70754576e-01
 -1.28482687e+00  2.15709

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('driver', 0.7645907402038574),
 ('cars', 0.7256535291671753),
 ('motorcycle', 0.7231885194778442),
 ('taxi', 0.7163602113723755),
 ('truck', 0.7151368856430054),
 ('vehicle', 0.6943105459213257),
 ('glider', 0.676424503326416),
 ('racing', 0.6492857933044434),
 ('passenger', 0.6429322957992554),
 ('automobile', 0.6412168741226196)]

#### 確認例題
日本語コーパス(ja.text8.zip)をダウンロードし、同様の処理をしてみましょう。

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip
!unzip ja.text8.zip

--2020-10-29 13:06:18--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.4.150
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.4.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33905114 (32M) [application/zip]
Saving to: ‘ja.text8.zip.1’

ja.text8.zip.1      100%[===================>]  32.33M  41.3MB/s    in 0.8s    

2020-10-29 13:06:19 (41.3 MB/s) - ‘ja.text8.zip.1’ saved [33905114/33905114]

Archive:  ja.text8.zip
replace ja.text8? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: ja.text8                


In [ ]:
import logging
from gensim.models.word2vec import Word2Vec, Text8Corpus

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = Text8Corpus('ja.text8')
model = Word2Vec(sentences, size=100)

model.save('model.ja.bin')

2020-10-29 13:06:37,472 : INFO : collecting all words and their counts
2020-10-29 13:06:37,478 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-29 13:06:43,369 : INFO : collected 290811 word types from a corpus of 16900026 raw words and 1691 sentences
2020-10-29 13:06:43,370 : INFO : Loading a fresh vocabulary
2020-10-29 13:06:43,609 : INFO : effective_min_count=5 retains 75187 unique words (25% of original 290811, drops 215624)
2020-10-29 13:06:43,610 : INFO : effective_min_count=5 leaves 16577418 word corpus (98% of original 16900026, drops 322608)
2020-10-29 13:06:43,852 : INFO : deleting the raw counts dictionary of 290811 items
2020-10-29 13:06:43,866 : INFO : sample=0.001 downsamples 34 most-common words
2020-10-29 13:06:43,868 : INFO : downsampling leaves estimated 11431523 word corpus (69.0% of prior 16577418)
2020-10-29 13:06:44,169 : INFO : estimated required memory for 75187 words and 100 dimensions: 97743100 bytes
2020-10-29 13:06:44,170 : 

In [ ]:
model_ja = Word2Vec.load('model.ja.bin')

print(model_ja.wv['犬'])
print(model_ja.wv['犬'].shape)
model_ja.wv.most_similar(['車'])

2020-10-29 13:44:32,903 : INFO : loading Word2Vec object from model.ja.bin
2020-10-29 13:44:33,744 : INFO : loading wv recursively from model.ja.bin.wv.* with mmap=None
2020-10-29 13:44:33,745 : INFO : setting ignored attribute vectors_norm to None
2020-10-29 13:44:33,745 : INFO : loading vocabulary recursively from model.ja.bin.vocabulary.* with mmap=None
2020-10-29 13:44:33,747 : INFO : loading trainables recursively from model.ja.bin.trainables.* with mmap=None
2020-10-29 13:44:33,748 : INFO : setting ignored attribute cum_table to None
2020-10-29 13:44:33,749 : INFO : loaded model.ja.bin
2020-10-29 13:44:33,931 : INFO : precomputing L2-norms of word weight vectors


[-0.83047616  0.7002652   1.8226818  -1.5889463  -1.3526665  -1.01874
  1.4335294   1.3037403   0.93054587  2.0960686  -0.81986654  1.5785731
  4.042562    0.4744235   1.4421889  -0.04492768 -0.6082116   2.5076668
  0.02987989  1.107785    0.23729905  0.08941399 -1.1753266  -1.5561072
 -0.0824669   1.3361316  -1.0303357   1.5947185   1.255065   -0.5746231
 -1.1723132   0.7795273   0.0620449   0.81751496 -1.0895228   0.00462517
  0.27096602  0.07449397  1.8582736   1.7072538   0.82855135  0.9623953
  0.61951864  0.10411737 -0.5599804  -0.12124187 -2.9421556   0.04146991
 -0.61044574  1.0615283  -0.66029936  0.2995721   0.11796718 -0.6701717
  0.01056705 -0.41598317 -1.0229447   1.5194993  -0.41964298 -2.4524255
 -0.6570432  -0.5400708  -1.1838228   0.16796957  1.2917123  -0.61429083
 -1.3051951  -0.32676408  0.27216005 -1.0149709  -1.9711257   0.03284217
 -1.0621338  -0.29972205  0.11460214 -0.3837586  -1.0412369  -0.01990454
 -0.37102276  1.503586    1.2830892  -1.5605084  -0.24049373 

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('車両', 0.8276019096374512),
 ('客車', 0.7826144695281982),
 ('貨車', 0.7474372982978821),
 ('車体', 0.7086012959480286),
 ('台車', 0.7060954570770264),
 ('気動車', 0.6966443061828613),
 ('寝台', 0.6952678561210632),
 ('電車', 0.6818050742149353),
 ('モハ', 0.638667106628418),
 ('全車', 0.638338565826416)]

### ベクトル空間モデル


*   情報検索を行うためのアルゴリズム
*   1970年頃から研究が始まる
 * Salton先生等のSMARTシステムが有名

* ベクトル空間上に検索対象データ，検索語をそれぞれベクトルで表現して配置する．
* 検索対象データ，検索語の類似度をベクトル計算（コサイン，内積，距離等）により求める。
<img src='https://drive.google.com/uc?export=view&id=14u9m4dFPDTmMUa4sMLMErm6CwViEiqpX' width='30%'>


### 文書単語行列
* 文書(d1～dm)中に出現する単語集合(t1～tn)によって，特徴付けられた行列

<img src='https://drive.google.com/uc?export=view&id=1y4V0uPCEPcm0JH_v0-NHWtz0SW4_bGkc' width='30%'>

<br>

* 文書単語行列の例:

<img src='https://drive.google.com/uc?export=view&id=1tEOkD8_BnWLuZxf9iiD1zlVCl8Zn7DAv' width='50%'>

### コサイン計算

<img src='https://drive.google.com/uc?export=view&id=1a226Fn74_d-un3DM-Z_d41yWixYQiYFy' width='60%'>

分散表現ベクトルのコサイン尺度を計算する関数cosine_sim()を作成し、「car」と「truck」の分散表現ベクトルのコサイン尺度を計算してみます。

In [ ]:
def cosine_sim(w1, w2):
  cosine_value = np.dot(model.wv[w1], model.wv[w2]) / (np.linalg.norm(model.wv[w1]) * np.linalg.norm(model.wv[w2]))
  
  return cosine_value

print (cosine_sim('car','truck'))

0.7151368


コサイン尺度は0.7151368と出力され、先ほどの「model.wv.most_similar(['car'])」の結果と同じ値が算出されていることが確認できます。

[('driver', 0.7645907402038574), <br>
 ('cars', 0.7256535291671753), <br>
 ('motorcycle', 0.7231885194778442), <br>
 ('taxi', 0.7163602113723755), <br>
 ('truck', 0.7151368856430054), <br>
 ('vehicle', 0.6943105459213257), <br>
...]

#### 確認例題
コサイン計算プログラムを利用して、「車」と「電車」の分散表現ベクトルのコサイン尺度を計算してみましょう。

### 文書分類



### 参考文献
* https://code.google.com/archive/p/word2vec/